In [12]:
# import relevant packages
from collections import defaultdict
import pandas as pd
import os
import numpy as np

In [13]:
# read data dictionary 
data_dict = pd.read_csv("data_dictionary.csv")

In [14]:
filepath = "/Volumes/SRUTI/release_08-19-2020/training/"

In [15]:
# list files in training set
os.listdir(filepath)

['observation.csv',
 'device_exposure.csv',
 'visit_occurrence.csv',
 'goldstandard.csv',
 'procedure_occurrence.csv',
 'person.csv',
 'measurement.csv',
 'location.csv',
 'drug_exposure.csv',
 'condition_occurrence.csv',
 'observation_period.csv',
 '._observation.csv',
 '._device_exposure.csv',
 '._visit_occurrence.csv',
 '._goldstandard.csv',
 '._procedure_occurrence.csv',
 '._person.csv',
 '._measurement.csv',
 '._location.csv',
 '._drug_exposure.csv',
 '._condition_occurrence.csv',
 '._observation_period.csv']

In [16]:
# read all csv files
goldstandard = pd.read_csv(filepath+"goldstandard.csv")
person = pd.read_csv(filepath+"person.csv")
measurement = pd.read_csv(filepath+"measurement.csv")
condition_occurrence = pd.read_csv(filepath+"condition_occurrence.csv")

### Exploratory data analysis

In [17]:
data_dict.head()

,concept_id,concept_name,table
0,22274,Neoplasm of uncertain behavior of larynx,condition_occurrence
1,22281,Sickle cell-hemoglobin SS disease,condition_occurrence
2,22288,Hereditary elliptocytosis,condition_occurrence
3,22340,Esophageal varices without bleeding,condition_occurrence
4,22350,Edema of larynx,condition_occurrence


In [18]:
data_dict.table.unique()

array(['condition_occurrence', 'observation', 'drug_exposure',
       'measurement', 'visit_occurrence', 'procedure_occurrence',
       'device_exposure'], dtype=object)

In [97]:
# convert to a dict for easy lookup
concept_id_map = dict(zip(data_dict.concept_id,data_dict.concept_name))

In [20]:
from datetime import datetime

def total_hours(s1, s2, FMT):
    tdelta = datetime.strptime(s2, FMT) - datetime.strptime(s1, FMT)
    # return time difference in hours
    return tdelta.total_seconds()/3600

#### Gold standard

Description: The goldstandard file will be available to models while they train and can be used designate true positive and true negative patients. This file stores the true status of the patients in relation to the question being asked. For instance, for question 1, the status is a binary declaring whether a patient tested positive (1.0) or negative (0.0).

Number of Rows: Same as person table

In [21]:
goldstandard.head()

,person_id,status
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [22]:
goldstandard.status.value_counts()

0.0    44947
1.0     4944
Name: status, dtype: int64

In [23]:
# this will be target y to predict
y = np.array(goldstandard.status)

------------

#### Person

Description: The PERSON table stores demographic information about all the patients in the repository. The Person table contains records that uniquely identify each patient in the data.

Number of Rows: 9,000

In [24]:
person.head()

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,0,8507,1948,3,7,1948-03-07,8527,38003563,6977,NaN,NaN,NaN,M,0,NaN,30,NaN,28
1,1,8507,1932,9,25,1932-09-25,8515,38003564,1605,NaN,NaN,NaN,M,0,NaN,10,NaN,29
2,2,8532,1963,6,2,1963-06-02,8527,38003564,2242,NaN,NaN,NaN,F,0,NaN,30,NaN,29
3,3,8532,1989,1,5,1989-01-05,8527,38003564,97,NaN,NaN,NaN,F,0,NaN,30,NaN,29
4,4,8507,1951,7,25,1951-07-25,8527,38003563,3915,NaN,NaN,NaN,M,0,NaN,30,NaN,28


In [25]:
person.gender_concept_id.value_counts()

8532    27130
8507    22761
Name: gender_concept_id, dtype: int64

In [26]:
person.drop(columns=['gender_concept_id', 'year_of_birth', 'month_of_birth',
       'day_of_birth', 'race_concept_id',
       'ethnicity_concept_id', 'provider_id', 'care_site_id',
       'person_source_value',
       'gender_source_concept_id', 'race_source_value',
       'ethnicity_source_value'], inplace=True)

In [27]:
person.head()

,person_id,birth_datetime,location_id,gender_source_value,race_source_concept_id,ethnicity_source_concept_id
0,0,1948-03-07,6977,M,30,28
1,1,1932-09-25,1605,M,10,29
2,2,1963-06-02,2242,F,30,29
3,3,1989-01-05,97,F,30,29
4,4,1951-07-25,3915,M,30,28


In [28]:
# what are all the races?
person.race_source_concept_id.value_counts()

30    22582
13     9134
10     9124
26     9051
Name: race_source_concept_id, dtype: int64

In [29]:
# what are all the genders?
person.gender_source_value.value_counts()

F    27130
M    22761
Name: gender_source_value, dtype: int64

In [30]:
# what are all the ethnicities?
person.ethnicity_source_concept_id.value_counts()

29    27326
19    13590
28     8975
Name: ethnicity_source_concept_id, dtype: int64

In [31]:
person.rename(columns={"gender_source_value": "gender", "race_source_concept_id": "race", "ethnicity_source_concept_id": "ethnicity", "birth_datetime":"age", "location_id":"location"}, errors="raise", inplace=True)

In [32]:
person['age'] = [round(total_hours(s1=s1,s2='2021-02-04',FMT='%Y-%m-%d')/(24*365)) for s1 in person.age] # in years

In [33]:
person.head()

,person_id,age,location,gender,race,ethnicity
0,0,73,6977,M,30,28
1,1,88,1605,M,10,29
2,2,58,2242,F,30,29
3,3,32,97,F,30,29
4,4,70,3915,M,30,28


-----------

#### Measurement

Description: The MEASUREMENT table contains records of Measurement, i.e. structured values (numerical or categorical) obtained through systematic and standardized examination or testing of a Person or Person's sample. The MEASUREMENT table contains both orders and results of such Measurements as laboratory tests, vital signs, quantitative findings from pathology reports, etc.

Number of Rows: 6,376,000

Vocabulary: LOINC, SNOMED

In [34]:
measurement.head()

,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_time,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,unit_concept_id,range_low,range_high,provider_id,visit_occurrence_id,visit_detail_id,measurement_source_value,measurement_source_concept_id,unit_source_value,value_source_value
0,1,43584,3007220,2014-07-19,2014-07-19 00:27:00,2015-07-28,44818702,4172703.0,106.00,NaN,NaN,62.00,325.00,NaN,NaN,NaN,NaN,3007220,NaN,106.00
1,2,23570,3022192,2011-10-24,2011-10-24 00:27:00,2017-04-03,44818702,4172703.0,137.00,NaN,8840.0,NaN,150.00,NaN,NaN,NaN,NaN,3022192,mg/dL,137.00
2,3,15111,3017855,2018-08-15,2018-08-15 00:27:00,2017-10-16,44818702,4172703.0,0.01,NaN,NaN,0.26,1.65,NaN,NaN,NaN,NaN,3017855,NaN,0.01
3,4,37555,3013721,2018-11-10,2018-11-10 00:27:00,2018-12-09,44818702,4172703.0,16.00,NaN,8645.0,9.00,38.00,NaN,NaN,NaN,NaN,3013721,U/L,16.00
4,5,37443,3040131,2011-06-03,2011-06-03 00:27:00,2019-07-25,44818702,4172703.0,2.03,NaN,NaN,0.90,2.50,NaN,NaN,NaN,NaN,3040131,NaN,2.03


In [35]:
def cutoff_time(s1):
    FMT='%Y-%m-%d'
    s2='2020-01-01'
    if datetime.strptime(s2, FMT) > datetime.strptime(s1, FMT): 
        return False #False if latest measurement date before 1st January 2020
    else:
        return True #True if latest measurement date after 1st January 2020

In [101]:
measurement_feature = ['3027018', '3020891', '3012888', '3004249', '4196147', '3016502', '3018465', '3024928', '3004327', '3013650',
                       '3009932', '3015242','3022250', '42870366', '3024929', '436238', '3026910', '3033575', '3013429', '3023314']

In [37]:
measurement.dropna(subset = ['measurement_concept_id', 'person_id', 'measurement_date', 'value_as_number'], inplace=True)

In [38]:
measurement = measurement[measurement.measurement_concept_id.isin(measurement_feature)]

In [39]:
measurement['2020_visit'] = measurement.apply(lambda x: cutoff_time(str(x['measurement_date'])),axis=1)

In [40]:
measurement.dropna(subset = ['measurement_concept_id', 'person_id', 'measurement_date', 'value_as_number'], inplace=True)

In [41]:
measurement = measurement[measurement['2020_visit'] == True]

In [44]:
measurement.shape

(176406, 8)

In [42]:
measurement.drop(columns=['measurement_id', 
       'measurement_datetime', 'measurement_time',
       'measurement_type_concept_id', 'operator_concept_id',
       'value_as_concept_id', 'unit_concept_id',
       'provider_id', 'visit_occurrence_id', 'visit_detail_id',
       'measurement_source_value', 'measurement_source_concept_id', 'value_source_value', '2020_visit'], errors='ignore', inplace=True)

In [43]:
measurement.head()

,person_id,measurement_concept_id,measurement_date,value_as_number,range_low,range_high,unit_source_value
5,3508,3004249,2020-03-12,133.0,77.0,126.0,mmHg
99,39712,3024929,2020-02-05,50.0,150.0,400.0,10*3/uL
192,2609,3027018,2020-04-01,119.0,70.0,110.0,bpm
567,46594,3015242,2020-03-31,10.0,20.0,230.0,ng/mL
668,24399,3024929,2020-02-24,35.0,150.0,400.0,10*3/uL


-----------

#### Condition Occurance

Description: The 'Drug' domain captures records about the utilization of a Drug when ingested or otherwise introduced into the body. A Drug is a biochemical substance formulated in such a way that when administered to a Person it will exert a certain physiological effect. Drugs include prescription and over-the-counter medicines, vaccines, and large-molecule biologic therapies. Radiological devices ingested or applied locally do not count as Drugs. Drug Exposure is inferred from clinical events associated with orders, prescriptions written, pharmacy dispensings, procedural administrations, and other patient-reported information.

Number of Rows: 6,582,000

Vocabulary: RxNorm

RxNorm is the attempt to unify the reference standard for medications and drugs. RxNorm incorporates other vocabularies, linking them together to unify drug referencing. 

In [44]:
condition_occurrence.head()

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id
0,1,36715,436659,2017-01-08,2017-01-08,2017-01-17,2017-01-17,32020,NaN,NaN,NaN,NaN,NaN,35206695,ENCOUNTER/ORDER SUMMARY,4033240
1,2,21628,4115171,2016-03-18,2016-03-18,2016-03-23,2016-03-23,32019,NaN,NaN,NaN,NaN,NaN,45533939,CHARGES,4230359
2,3,41547,78232,2011-02-13,2011-02-13,2011-02-22,2011-02-22,43542353,NaN,NaN,NaN,NaN,NaN,45591807,ORDER,4033240
3,4,11546,439846,2017-10-04,2017-10-04,2017-10-05,2017-10-05,32019,NaN,NaN,NaN,NaN,NaN,35207792,CHARGES,4230359
4,5,43031,78097,2017-10-01,2017-10-01,2017-10-10,2017-10-10,32019,NaN,NaN,NaN,NaN,NaN,45552285,BILLING,4230359


In [45]:
condition_feature = ['254761','4305080','442752','378253','259153','196523',
                     '31967','27674','45757468','441408','434490','4168213',
                     '77670','4223659','315078','437663','439937']

In [46]:
condition_occurrence = condition_occurrence[condition_occurrence.condition_concept_id.isin(condition_feature)]

In [47]:
condition_occurrence['condition_duration'] = [total_hours(s1,s2,FMT='%Y-%m-%d') for s1,s2 in zip(condition_occurrence.condition_start_date, condition_occurrence.condition_end_date)]

In [48]:
condition_occurrence['2020_visit'] = condition_occurrence.apply(lambda x: cutoff_time(str(x['condition_start_date'])),axis=1)

In [49]:
condition_occurrence.dropna(subset = ['condition_concept_id', 'person_id', 'condition_start_date'], inplace=True)

In [50]:
condition_occurrence['2020_visit'].value_counts()

False    453868
True      24985
Name: 2020_visit, dtype: int64

In [51]:
condition_occurrence = condition_occurrence[condition_occurrence['2020_visit'] == True]

In [52]:
condition_occurrence.drop(columns=['condition_occurrence_id','condition_start_datetime',
       'condition_end_date', 'condition_end_datetime',
       'condition_type_concept_id', 'stop_reason', 'provider_id',
       'visit_occurrence_id', 'visit_detail_id', 'condition_source_value',
       'condition_source_concept_id', 'condition_status_source_value',
       'condition_status_concept_id', '2020_visit'], errors='ignore', inplace=True)

In [53]:
condition_occurrence.head()

,person_id,condition_concept_id,condition_start_date,condition_duration
777,25489,27674,2020-02-23,24.0
828,42474,77670,2020-01-20,216.0
1290,12263,254761,2020-07-16,24.0
1559,32650,77670,2020-03-26,192.0
2537,38150,77670,2020-04-27,216.0


---------

In [54]:
person.dropna(subset=['person_id', 'age', 'gender', 'location', 'gender', 'race', 'ethnicity'], inplace=True)

In [55]:
person.drop_duplicates(['person_id'], inplace=True)

In [84]:
rows = len(person.person_id)
columns = len(measurement_feature) + len(condition_feature)
feature_matrix = np.zeros((rows,columns))

In [86]:
for i,concept_id in enumerate(condition_feature):
    filtered = condition_occurrence[condition_occurrence.condition_concept_id==int(concept_id)]
    person_id_subset = set(filtered.person_id)
    for pid in person_id_subset:
        feature_matrix[pid,i] = 1

In [87]:
feature_matrix.shape

(49891, 37)

In [88]:
np.array(person)

array([[0, 73, 6977, 'M', 30, 28],
       [1, 88, 1605, 'M', 10, 29],
       [2, 58, 2242, 'F', 30, 29],
       ...,
       [49888, 19, 7694, 'M', 26, 19],
       [49889, 76, 7150, 'F', 30, 29],
       [49890, 31, 270, 'M', 30, 28]], dtype=object)

In [89]:
for i,concept_id in enumerate(measurement_feature, len(condition_feature)):
    filtered = measurement[measurement.measurement_concept_id==int(concept_id)]
    for pid in person.person_id:
        try:
            sub = filtered[filtered.person_id==pid]
            dates = sub.measurement_date
            dates = [datetime.strptime(s, '%Y-%m-%d') for s in dates]
            dates.sort() # sort dates in ascending order
            last_date = dates[-1] # last date
            sub = sub[sub.measurement_date==last_date.strftime('%Y-%m-%d')]
            feature_matrix[pid,i] = sub.value_as_number.values[0]
        except:
            feature_matrix[pid,i] = np.nan

In [ ]:
start_from = len(condition_feature)

In [ ]:
for i,concept_id in enumerate(measurement_feature, start_from):
    filtered = measurement[measurement.measurement_concept_id==int(concept_id)]
    for pid in person.person_id:
        try:
            sub = filtered[filtered.person_id==pid]
            if concept_id in down_regulated:
                if len([*filter(lambda x: x <= threshold[concept_id], sub.value_as_number.values)]) > 0
                    feature_matrix[pid,i] = 1
            else:
                if len([*filter(lambda x: x >= threshold[concept_id], sub.value_as_number.values)]) > 0
                    feature_matrix[pid,i] = 1
        except:
            pass

In [91]:
feature_matrix[:10,17:]

array([[     nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan, 2.09e+02,      nan,      nan,      nan,
             nan,      nan],
       [7.90e+01, 3.58e+01,      nan,      nan,      nan,      nan,
             nan,      nan, 7.60e-01,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan],
       [     nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan,      nan, 9.05e+00,      nan, 8.00e+00,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan],
       [     nan,      nan,      nan,      nan,      nan,      nan,
             nan, 5.90e+01,      nan,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan],
       [     nan,      nan,      nan,      nan,      nan,      nan,


In [102]:
column_names = [concept_id_map[int(concept_id)] for concept_id in condition_feature+measurement_feature]

In [104]:
feature_df = pd.DataFrame(feature_matrix, columns=column_names)

In [105]:
X = pd.concat([person, feature_df], axis=1)

In [106]:
X.head()

,person_id,age,location,gender,race,ethnicity,Cough,Abnormal breathing,Muscle pain,Headache,...,Eosinophils [#/volume] in Blood by Manual count,Ferritin [Mass/volume] in Serum or Plasma by Immunoassay,Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction,Fibrin D-dimer FEU [Mass/volume] in Blood by Immunoassay,Platelets [#/volume] in Blood by Automated count,Elevated levels of transaminase & lactic acid dehydrogenase,Gamma glutamyl transferase [Enzymatic activity/volume] in Serum or Plasma,Monocytes [#/volume] in Blood by Automated count,Basophils [#/volume] in Blood by Automated count,Hematocrit [Volume Fraction] of Blood by Automated count
0,0,73,6977,M,30,28,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,209.0,NaN,NaN,NaN,NaN,NaN
1,1,88,1605,M,10,29,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,58,2242,F,30,29,0.0,0.0,0.0,0.0,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,32,97,F,30,29,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,70,3915,M,30,28,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
X.isnull().sum()

person_id                                                                                                   0
age                                                                                                         0
location                                                                                                    0
gender                                                                                                      0
race                                                                                                        0
ethnicity                                                                                                   0
Cough                                                                                                       0
Abnormal breathing                                                                                          0
Muscle pain                                                                                                 0
Headache  

In [110]:
X

,person_id,age,location,gender,race,ethnicity,Cough,Abnormal breathing,Muscle pain,Headache,...,Eosinophils [#/volume] in Blood by Manual count,Ferritin [Mass/volume] in Serum or Plasma by Immunoassay,Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction,Fibrin D-dimer FEU [Mass/volume] in Blood by Immunoassay,Platelets [#/volume] in Blood by Automated count,Elevated levels of transaminase & lactic acid dehydrogenase,Gamma glutamyl transferase [Enzymatic activity/volume] in Serum or Plasma,Monocytes [#/volume] in Blood by Automated count,Basophils [#/volume] in Blood by Automated count,Hematocrit [Volume Fraction] of Blood by Automated count
0,0,73,6977,M,30,28,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,209.0,NaN,NaN,NaN,NaN,NaN
1,1,88,1605,M,10,29,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,58,2242,F,30,29,0.0,0.0,0.0,0.0,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,32,97,F,30,29,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,70,3915,M,30,28,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49886,49886,87,786,F,26,19,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,160.0,NaN,NaN,NaN,NaN,NaN
49887,49887,87,2089,F,26,19,0.0,0.0,0.0,0.0,...,NaN,NaN,147.0,NaN,NaN,NaN,NaN,0.48,NaN,NaN
49888,49888,19,7694,M,26,19,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN
49889,49889,76,7150,F,30,29,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
X.dropna(subset=['Ferritin [Mass/volume] in Serum or Plasma by Immunoassay', 
                 'Oxygen saturation in Arterial blood', 
                 'Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction'])

,person_id,age,location,gender,race,ethnicity,Cough,Abnormal breathing,Muscle pain,Headache,...,Eosinophils [#/volume] in Blood by Manual count,Ferritin [Mass/volume] in Serum or Plasma by Immunoassay,Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction,Fibrin D-dimer FEU [Mass/volume] in Blood by Immunoassay,Platelets [#/volume] in Blood by Automated count,Elevated levels of transaminase & lactic acid dehydrogenase,Gamma glutamyl transferase [Enzymatic activity/volume] in Serum or Plasma,Monocytes [#/volume] in Blood by Automated count,Basophils [#/volume] in Blood by Automated count,Hematocrit [Volume Fraction] of Blood by Automated count
456,456,66,6917,F,30,28,0.0,0.0,0.0,0.0,...,NaN,253.0,232.0,NaN,110.0,NaN,NaN,NaN,NaN,NaN
486,486,89,7816,F,30,19,0.0,0.0,0.0,0.0,...,NaN,169.0,277.0,NaN,NaN,NaN,NaN,NaN,NaN,29.0
615,615,24,3768,M,26,19,0.0,0.0,0.0,0.0,...,NaN,71.0,373.0,0.42,NaN,NaN,NaN,NaN,NaN,NaN
1377,1377,63,4256,M,30,29,0.0,0.0,0.0,0.0,...,NaN,1399.0,629.0,NaN,NaN,NaN,NaN,0.54,NaN,NaN
1460,1460,64,1097,F,13,29,0.0,0.0,0.0,0.0,...,NaN,168.0,297.0,NaN,NaN,NaN,NaN,1.06,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46977,46977,51,467,F,26,19,0.0,0.0,0.0,0.0,...,NaN,39.0,229.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47393,47393,84,3226,F,10,29,0.0,0.0,0.0,0.0,...,NaN,105.0,412.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47546,47546,29,3894,F,30,28,0.0,0.0,0.0,0.0,...,NaN,78.0,827.0,0.37,NaN,NaN,NaN,NaN,NaN,NaN
47648,47648,17,5289,F,13,29,0.0,0.0,0.0,0.0,...,NaN,44.0,259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
